# 第9回 Monte-Carlo Tree Search 2

## Monte-Carlo Tree Search

### 前回振り返り: 原始モンテカルロ囲碁は何が弱いか

前回説明したアルゴリズムは、各選択肢から何回か Rollout (Playout) を行い、
報酬の期待値が高い選択肢を選ぶという物だった。
それぞれの選択肢を試す回数は均等でも良い。
しかし改善方法として、バンディット問題を応用して有望な選択肢に計算時間を投入する方法を紹介した。

残念ながらこのアルゴリズムには弱点があり、例えばパズルやゲームに適用した場合、特に計算時間を無限に投入しても最善の選択肢を選ぶ保証がない。
（保証がない、というのは控えめな表現で、偶然最善の選択肢を選ぶ事もある、というぐらいに考えると良い。）
その理由は、簡単に言えば Rollout (playout) ではお互いにランダムに選択肢を選ぶからである。
お互いが完全にランダムに Rollout を行う場合、相手のミスに期待する選択肢の評価が高くなる。

<img src="better_mcgo.png" width=300>

たとえば将棋なら飛車の頭に歩を打つと、50%ぐらいの確率で飛車を取れる。
（チェスならクイーンを直接取りに行く様な手、囲碁なら「アタリ」をかける手、などになる。）
実際にこの手法をそのままナイーブに使ったプログラムに将棋を指させると、飛車の頭に駒を打って行って、タダでとられることを繰り返すようになる。

### 木探索への改良

簡単な工夫を加えることによって上述の弱点を解消できる。
それは、以下の図の様に有望な手を展開して1つ深く読むということだ。
バンディット問題などを応用して有望な手ほど多くの Rollout が実行されるようにする。
それに加えて、ある節点からの Rollout の回数が閾値に達したら、その節点を一段展開する。
こうしてRollout を開始する節点が一段深くなる。
これを繰り返して有望な方向へ、探索木を伸ばしていくのが **Monte-Carlo Tree Search** である。

<img src="mcts.png" width=300>

> R. Coulom, "Efficient Selectivity and Backup Operators in Monte-Carlo Tree Search", Computers and Games 2006.

### 探索の動作

<img src="mcts_steps.png" width=800>

MCTS は上図に示すように4ステップを繰り返す。
1. selection: root 節点から最も有望な節点を選び leaf 節点まで到達する
1. expansion: もし leaf 節点で既に閾値以上の回数 rollout が行われていたら子節点を展開し、子節点のどれかを leaf 節点とする
1. simulation: 上のステップで選択された leaf 節点から Rollout を実行し、報酬を得る
1. backpropagation: 得られた報酬を元に、経路上の節点の訪問回数と報酬を更新する

制限時間が来るか、決められた回数 Rollout が実行されたら終了するというのが一般的である。
以下に疑似コードの一例を示す。
関数 MCTS の引数は開始節点であり、返値としては一番有望な選択肢を返す。

```
fun MCTS(root_state) {
  create root_node from root_state
  add root_node to tree
  while (time_is_remaining) {
    path = Selection(root_node)
    leaf = last element of path
    child = Expansion(leaf)
    reward = Simulation(child)
    BackPropagation(reward, child)
  }
  return the branch with the highest number of rollouts
}
```

この例では、関数 Selection は root_node から leaf までの節点のリスト、つまり現時点で最も有望な選択肢をたどった場合の経路を返している。
SelectBestChild の内容は示していないが、ここでは例えば前回説明した Upper Confidence Bound に基づく UCB1 の式を使う。
UCB1に基づいて Selection を行うMCTS を **Upper Confindence bound applied to Trees (UCT)** アルゴリズムと言う。

```
fun Selection(root_node) {
  path = [root_node] 
  node = root_node
  while (node is not leaf) {
    child_node = SelectBestChild(node)
    path.append[child_node]
  }
  return path
}
```

関数 Expansion は、leaf 節点を受け取って、条件を満たしている場合に展開をする。
node は訪問回数 visit と報酬の合計 reward を持つとする。
訪問回数が **expand_threshold** 以上の場合、節点を展開して新たに作成された子節点のどれかを返す。

```
fun Expansion(node) {
  if (node.visits >= expand_threshold) {
    create children of node and add to tree
    return one of the children
  }
  else {
    return node
  }
}
```

Simulationは Rollout (playout) のことである。
子節点をランダムにたどりながら末端まで木をたどり、そこでスコアを計算して報酬として返す。
（なお、ここまでの説明では末端までたどることを前提としているが、実際には必ずしもその必要はない。
何らかの報酬を返すブラックボックスとみなして良い。）

```
fun Simulation(node) {
  while (node is not terminal) {
    child = SelectChildInRollout(node)
    node = child
  }
  reward = Score(node)
}
```

Selectionでたどった経路を逆向きにたどって、各節点の報酬と訪問回数を更新する。
なお、もし二人ゼロ和ゲームの場合には、1回上へ上がる度にスコアを反転すれば良い。
この例では、rewardの符号を反転しているが、これは勝/負/引き分けの報酬を +1/-1/0 などとした場合に対応する。
もし勝/負/引き分けの報酬が 1/0/0.5 などなら、反転するには reward = 1-reward とすれば良い。
（敵がいない場合はもちろんその必要はない。）

```
fun Backpropagation(leaf, reward) {
  node = leaf
  while (node is not root node) {
    node.visits += 1
    node.reward += reward
    node = leaf.parent
    reward = -reward
  }
}
```

以上の4ステップを繰り返した後、最終的には最も訪問回数の多かった選択肢を返すのが普通の方法である。
これを **robust max** と言う。
平均の報酬が最大の選択肢を選ぶよりは robust max の方が多くの場合に良い。

## MCTS の性質

### 最適性

UCB1を使ったUCTアルゴリズムの場合には、Rolloutの回数を無限大に増やすと、最善解に収束するという証明が与えられている。
より正確に言うと、Rollout回数が無限大になった場合、最善の選択肢が選ばれる確率が1に限りなく近づく。
別の言い方をすると、UCB1の式の第2項 (bias項) が0に収束する。

$ \frac{w_i}{s_i} + C \sqrt{ \frac{2 \ln t}{s_i} }$

($i$番目の枝の rollout が $s_i$ 回で$w_i$、また $t$ は親節点の rollout 回数の合計とする。)

> L. Kocsis, C. Szepesvári, "Bandit Based Monte-Carlo Planning", ECML 2006.


#### 利点

1. 評価関数が無くても動く。
    ただし、Rollout が完全にランダムだとそれほど強くない。対処法は次回説明する
1. 同じ枠組みで色々な問題に対応できる
    single agent (敵のいないケース、パズルや最短経路探索を含む)、二人ゲーム、多人数ゲームの全てで成功例がある
1. Anytime性がある (極端に Rollout に時間がかかる場合を除く)
1. 機械学習と組み合わせやすい。これも次回さらに詳しく説明する

#### 欠点

1. 苦手な形の木がある。
    確率的な動作をするアルゴリズムであるため、正解へ至る道が細く長いと発見するのに非常に長い時間がかかる。
1. Rollout が作りにくい場合がある。
    オセロ、3目並べなどはランダムな Rollout でも自然に終局する。将棋などではそのままでは難しい。
1. 合流への対処が難しい

<img src="dag.png" width=300>


## 第9回課題

- 9-1. 【selection と simulation】 3目並べについて、MCTSの Selection と Simulation を実装せよ。以下のコードを参考にしても良い。
- 9-2. 【backpropagation】 3目並べについて、MCTSの backpropagation を実装せよ。特に、報酬の反転や報酬の正負について、バグを入れやすいので注意すること。以下のコードを参考にしても良い。
- 9-3. 【MCTS3目並べ】 上記実装が終わったら、 rollout 回数を増減させて実行結果を観察せよ。また、exploration constant や expand threshold の値を変更して動作の違いを観察せよ。
- 9-4. 発展課題 【3目並べ対戦】 コードを少し変更し、自分自身と対戦可能にせよ。入力はテキストで手の座標を入れるなどの方法でも良い。出力もテキストでも良い。
- 9-5. 発展課題 【MCTS4目並べ】 重力付き4目並べ （Connect Four) について、同様にMCTSを実装せよ。また動作を観察せよ。さらに対戦可能とせよ。


### 参考コードのデータ構造の解説

class TicTacToeState が盤面の情報を保持する。
勝敗判定なども実装済みである（が、バグが入っている可能性はある）。
class TicTacToeNode が訪問回数 $s_i$ や累積の報酬 $w_i$を持つ。
報酬はこの例では勝/負/引き分けに対して+1/-1/0としている。

TicTacToeNodeからなる tree はハッシュテーブルに保存する。
node_id を key としてアクセスする。node_id は action (手) のtupleとして定義している。
action は 0--8 の数字とする。
また、 root node (初期盤面) の node_id は (0) とする。
root node から1手進めた局面のnode_id は (0, 0), (0, 1), ... ,(0, 8) となる。
要素が1個の tuple を初期化するための python 特有の書き方に注意。例えば 0 を唯一の要素とするサイズ1の tuple は以下の様に定義できる。

```Python
t = (0, )
```

関数MCTSの引数として Rollout の回数を指定している。また、exploration constant, expand threshold は class MCTS の変数として定義している。 

このコードは分かりやすさ、シンプルさを重視しているため、このままのコードを真似して大きな問題を解こうとすると速度もメモリ消費も無駄があるので注意すること。
たとえば上で説明しているハッシュキーについてはもっと効率的なテクニックがある。また、ハッシュテーブルに盤面自体を全て保存しているが、（3目並べなら問題ないが）これも省くことができる。

In [29]:
players_str = ['X', 'O']

class TicTacToeState:
    def __init__(self, board, player):
        '''
        initial board
        [[' ',' ',' '],
         [' ',' ',' '],
         [' ',' ',' ']]                           
        '''
        self.board = board
        self.player = player

    def show_board(self):
        for i in range(3):
            for j in range(3):
                print(self.board[i][j], end="")
            print('|')
        
    def make_next_state(self, move, player_id=None):
        '''
        return next state
        '''
        if player_id == None:
            player_id = self.player
        new_board = copy.deepcopy(self.board)
        x, y = move
        char = players_str[player_id]
        new_board[x][y] = char
        return TicTacToeState(new_board, 1-player_id)
    
    def is_terminal(self):
        '''
        return true if state is terminal
        '''
        if self.is_win(self.player):
            return True
        elif self.is_win(1-self.player):
            return True
        elif self.is_draw():
            return True
        return False
    
    def score(self, player_id = None):
        if player_id == None:
            player_id = self.player
        if self.is_win(player_id):
            return 1
        elif self.is_win(1 - player_id):
            return -1
        elif self.is_draw():
            return 0
        assert(False)
        
    def is_draw(self):
        for i in range(3):
            for j in range(3):
                if self.board[i][j]==' ':
                    return False
        return True

    def is_win(self, player_id = None):
        if player_id == None:
            player_id = self.player
        char = players_str[player_id]
        for i in range(3):
            win_flag = True
            for j in range(3):
                if self.board[i][j]!=char:
                    win_flag = False
                    break
            if win_flag:
                return True
            
        for i in range(3):
            win_flag = True
            for j in range(3):
                if self.board[j][i]!=char:
                    win_flag = False
                    break
            if win_flag:
                return True

        win_flag = True
        for i in range(3):
            if self.board[i][i]!=char:
                win_flag = False
                break
        if win_flag:
            return True

        win_flag = True
        for i in range(3):
            if self.board[2-i][i]!=char:
                win_flag = False
                break
        if win_flag:
            return True

        return False


class TicTacToeNode:
    def __init__(self, state, node_id, player, w, s):
        '''
        state = board state
        player: player id 0 ("X") or 1 ("O")
        w: total reward
        s: number of visits
        '''
        self.state = state
        self.node_id = node_id
        self.player=player
        self.w = w # total reward
        self.s = s # number of visits
        self.expanded = False
        self.nu_child = 0
        self.children = []

    def is_root_node(self):
        if self.node_id == (0,):
            return True
        else:
            return False

    def print(self):
        print(self.node_id, self.w, self.s)
        #self.state.show_board()
        #for c in self.children:
        #    print(c)

# 課題1 and 課題2

In [126]:
import copy
import random
import numpy as np

class MCTS:
    def __init__(self, expand=1,exploration=1,nu_rollout=10):
        self.tree = {}
        # 以下の定数は適当に変更して動作を観察すると良い
        self.expand_threshold = expand
        self.exploration_constant = exploration
        self.nu_rollout = nu_rollout

    def selection(self, root_node):
        node = root_node
        node_id = root_node.node_id
        path = [node_id]
        '''
        例えば、ここを実装せよ
        '''
        player=root_node.player
        while node.nu_child>0:
            value=[]
            for i in node.children:
                j=self.tree[i]
                value.append(self.UCB1(j.w,j.s,node.s))
            if root_node.player==player:    
                num=np.argmax(value)
            else:
                num=np.argmin(value)
            path.append(node.children[num])
            node=self.tree[node.children[num]]
            player=1-player
        return path

    def UCB1(self, w, s, t):
        if s == 0:
            return 10000
        mean_reward = w/s
        bias_value = np.sqrt(np.log(t)/s)
        ucb1_value = mean_reward + self.exploration_constant * bias_value
        return ucb1_value
        
    def expansion(self, node):
        '''
        generate all children (legal moves)
        '''
        next_player = 1 - node.state.player

        if node.s < self.expand_threshold and not node.is_root_node():
            return node
        
        num = 0
        for i in range(3):
            for j in range(3):
                if node.state.board[i][j]==' ':
                    move = (i, j)
                    action_id = i*3 + j
                    num += 1
                    child_id = node.node_id + (action_id,)
                    node.children.append(child_id)
                    child_state = node.state.make_next_state(move)
                    child_node = TicTacToeNode(child_state, child_id, next_player, 0, 0)
                    self.tree[child_id] = child_node
                    
        node.expanded = True
        node.nu_child = num
        if len(node.children) == 0:
            return node
        leaf_id = random.choice(node.children)
        return self.tree[leaf_id]
        

    def simulation(self, node,root_player):
        state = node.state
        '''
        前回の課題 8-3 を参考にして良いが、データ構造が変わっていることに注意
        '''
        while not state.is_terminal():
            blank=[]
            for i in range(3):
                for j in range(3):
                    if node.state.board[i][j]==' ':
                        blank.append([i,j])
            m=random.choice(blank)
            state=state.make_next_state(m)
        return state.score(root_player)
            

    def backpropagation(self, path, reward):
        '''
        以下を実装せよ
        '''
        for i in range(len(path)-1,-1,-1):
            node=self.tree[path[i]]
            node.s+=1
            node.w+=reward
            

    def start_search(self, root_board, player):
        root_node_id = (0, ) # 要素が1個のtupleを作っている
        root_state = TicTacToeState(root_board, player)
        root_node = TicTacToeNode(root_state, root_node_id, player, 0, 0)
        self.tree[root_node_id] = root_node
        
        if root_node.state.is_terminal():
            score = root_node.state.score()
            return None, score

        for n in range(self.nu_rollout):
            path = self.selection(root_node)
            leaf_node_id = path[-1]
            leaf_node = self.tree[leaf_node_id]
            leaf_node = self.expansion(leaf_node)
            reward = self.simulation(leaf_node,player)
            self.backpropagation(path, reward)

        best_value = -10000
        best_child_id = None
        assert(root_node.nu_child >= 1)
        for child_id in root_node.children:
            child_node = self.tree[child_id]
            value = child_node.w
            print(child_id, (child_node.w/child_node.s))
            if value > best_value:
                best_value = value
                best_child_id = child_id

        return best_child_id, (root_node.w/root_node.s)

root_board = [[' ',' ',' '],
              [' ',' ',' '],
              [' ',' ',' ']]                           

search = MCTS(nu_rollout=10)
search.start_search(root_board, 0)

(0, 0) -1.0
(0, 1) -1.0
(0, 2) 1.0
(0, 3) 1.0
(0, 4) 1.0
(0, 5) 1.0
(0, 6) 1.0
(0, 7) 1.0
(0, 8) -1.0


((0, 2), 0.4)

In [127]:
root_board = [[' ',' ','X'],
              [' ','O',' '],
              ['X',' ','O']]   

search = MCTS(nu_rollout=1000)
search.start_search(root_board, 0)

(0, 0) 0.983789260385005
(0, 1) -0.5
(0, 3) -1.0
(0, 5) -1.0
(0, 7) -0.5


((0, 0), 0.962)

# 課題3

In [128]:
root_board = [[' ',' ',' '],
              [' ',' ',' '],
              [' ',' ',' ']]                           

search = MCTS(nu_rollout=100)
search.start_search(root_board, 0)

(0, 0) -1.0
(0, 1) -0.2
(0, 2) -0.3333333333333333
(0, 3) -0.3333333333333333
(0, 4) 0.6923076923076923
(0, 5) -0.3333333333333333
(0, 6) 0.3076923076923077
(0, 7) -0.3333333333333333
(0, 8) -1.0


((0, 4), 0.41)

In [129]:
root_board = [[' ',' ',' '],
              [' ',' ',' '],
              [' ',' ',' ']]                           

search = MCTS(nu_rollout=1000)
search.start_search(root_board, 0)

(0, 0) 0.9255441008018328
(0, 1) 0.2857142857142857
(0, 2) 0.25
(0, 3) -0.2
(0, 4) 0.35
(0, 5) 0.09090909090909091
(0, 6) 0.25
(0, 7) 0.2857142857142857
(0, 8) 0.4666666666666667


((0, 0), 0.846)

In [130]:
root_board = [[' ',' ',' '],
              [' ',' ',' '],
              [' ',' ',' ']]                           

search = MCTS(expand=5,exploration=1,nu_rollout=100)
search.start_search(root_board, 0)

(0, 0) 0.16666666666666666
(0, 1) -0.5
(0, 2) -0.2
(0, 3) -0.2
(0, 4) -0.2
(0, 5) 0.29411764705882354
(0, 6) 0.42857142857142855
(0, 7) 0.0
(0, 8) 0.2


((0, 6), 0.16)

In [131]:
root_board = [[' ',' ',' '],
              [' ',' ',' '],
              [' ',' ',' ']]                           

search = MCTS(expand=10,exploration=1,nu_rollout=100)
search.start_search(root_board, 0)

(0, 0) -0.2
(0, 1) -0.3333333333333333
(0, 2) -0.3333333333333333
(0, 3) 0.25
(0, 4) 0.6976744186046512
(0, 5) 0.38461538461538464
(0, 6) 0.38461538461538464
(0, 7) -0.3333333333333333
(0, 8) 0.25


((0, 4), 0.41)

In [132]:
root_board = [[' ',' ',' '],
              [' ',' ',' '],
              [' ',' ',' ']]                           

search = MCTS(expand=1,exploration=5,nu_rollout=100)
search.start_search(root_board, 0)

(0, 0) 0.5
(0, 1) -0.125
(0, 2) 0.1111111111111111
(0, 3) 0.5
(0, 4) 0.2
(0, 5) 0.5
(0, 6) 0.2727272727272727
(0, 7) 0.5384615384615384
(0, 8) 0.3333333333333333


((0, 7), 0.35)

In [133]:
root_board = [['X','X',' '],
              [' ','O',' '],
              [' ','O',' ']]                           

search = MCTS(expand=5,exploration=10,nu_rollout=1000)
search.start_search(root_board, 0)

(0, 2) 1.0
(0, 3) 0.7092511013215859
(0, 5) -0.20202020202020202
(0, 6) 0.5372340425531915
(0, 8) 0.35443037974683544


((0, 2), 0.624)

# 課題4

In [138]:
root_board = [[' ',' ',' '],
              [' ',' ',' '],
              [' ',' ',' ']]      
root=TicTacToeState(root_board,0)
player=0
while not root.is_terminal():
    search = MCTS(expand=20,exploration=1,nu_rollout=1000)
    step,prob=search.start_search(root.board, player)
    move=[step[1]//3,step[1]%3]
    print(step,move,prob)
    root=root.make_next_state(move, player_id=player)
    root.show_board()
    print('------------------------------------')
    player=1-player

(0, 0) 0.8142292490118577
(0, 1) 0.058823529411764705
(0, 2) 0.2571428571428571
(0, 3) 0.14285714285714285
(0, 4) 0.25
(0, 5) 0.16666666666666666
(0, 6) 0.35294117647058826
(0, 7) 0.2
(0, 8) -0.4
(0, 0) [0, 0] 0.672
X  |
   |
   |
------------------------------------
(0, 1) -1.0
(0, 2) -0.2222222222222222
(0, 3) -0.22580645161290322
(0, 4) -0.22580645161290322
(0, 5) -0.3333333333333333
(0, 6) 0.8665865384615384
(0, 7) -1.0
(0, 8) -0.175
(0, 6) [2, 0] 0.676
X  |
   |
O  |
------------------------------------
(0, 1) 0.2
(0, 2) 0.2
(0, 3) 0.42105263157894735
(0, 4) 0.9722838137472284
(0, 5) -0.2
(0, 7) 0.375
(0, 8) 0.6216216216216216
(0, 4) [1, 1] 0.918
X  |
 X |
O  |
------------------------------------
(0, 1) -1.0
(0, 2) -1.0
(0, 3) -0.6666666666666666
(0, 5) -1.0
(0, 7) -1.0
(0, 8) 0.9388379204892966
(0, 8) [2, 2] 0.904
X  |
 X |
O O|
------------------------------------
(0, 1) -0.5
(0, 2) -1.0
(0, 3) -0.3333333333333333
(0, 5) 0.18181818181818182
(0, 7) 0.9475890985324947
(0, 7) [2, 

# 課題5

In [2]:
import copy
import random
import numpy as np

class ConnectFour:
    def __init__(self):
        # 7x6
        self.players = ['X', 'O']
        
    def is_terminal(self, state):
        if self.is_draw(state):
            return True
        elif self.is_win(state,0) or self.is_win(state,1):
            return True
        return False
        
    def is_draw(self, state):
        if self.is_win(state,0) or self.is_win(state,1):
            return False
        for i in range(7):
            if state[i][0]==' ':
                return False
        return True

    def is_win(self, state, player_id):
        li=[]
        for i in range(7):
            for j in range(6):
                if state[i][j]==self.players[player_id]:
                    li.append([i,j])
        for i in li:
            if [i[0]+1,i[1]] in li and [i[0]+2,i[1]] in li and [i[0]+3,i[1]] in li:
                return True
            if [i[0],i[1]+1] in li and [i[0],i[1]+2] in li and [i[0],i[1]+3] in li:
                return True
            if [i[0]+1,i[1]+1] in li and [i[0]+2,i[1]+2] in li and [i[0]+3,i[1]+3] in li:
                return True
            if [i[0]-1,i[1]+1] in li and [i[0]-2,i[1]+2] in li and [i[0]-3,i[1]+3] in li:
                return True
        return False
    
    def reward(self,state):
        if self.is_draw(state):
            return 0
        elif self.is_win(state,0):
            return 1
        elif self.is_win(state,1):
            return -1
        
    def make_move(self, state, move, player_id):
        new_state = copy.deepcopy(state)
        for i in range(6):
            if state[move][i] != ' ':
                char = self.players[player_id]
                new_state[move][i-1] = char
                return new_state
        char = self.players[player_id]
        new_state[move][5] = char
        return new_state

            
    def board_blank(self,state):
        blank=[]
        for i in range(7):
            if state[i][0]==' ':
                blank.append(i)
        return blank
    
    def show_board(self,state):
        for i in state:
            print(i)

class mcts_node:
    def __init__(self,parent=None,move=None,state=None,player_id=None,game=None):
        self.childnode=[]
        self.move=move
        self.state=state
        self.visit=0
        self.value=0
        self.parent=parent
        self.player_id=player_id
        self.game=game
        if game.is_terminal(state):
            self.untried_node=[]
        else:
            self.untried_node=game.board_blank(state)
        
        
    def select_by_ucb(self):
        reward_li=[]
        for i in self.childnode:
            ucb=(i.value/i.visit)+np.sqrt(2*np.log(self.visit)/i.visit)
            reward_li.append(ucb)
        if self.player_id ==0:
            num=np.argmax(reward_li)
        else:
            num=np.argmin(reward_li)
        return self.childnode[num]
        
    
    def expand(self,move,state):
        child=mcts_node(parent=self,move=move,state=state,player_id=1-self.player_id,game=self.game)
        self.untried_node.remove(move)
        self.childnode.append(child)
        return child
        
        
    def backpropagation(self,reward):
        self.visit+=1
        self.value+=reward
        


def mcts(game,begin_state,iteration):
    mstnode=mcts_node(parent=None,move=None,state=begin_state,player_id=0,game=game)
    
    for i in range(iteration):
        node=mstnode
        state=begin_state
        id=mstnode.player_id

                
        while node.untried_node == [] and node.childnode !=[]:
            node=node.select_by_ucb()
            state=game.make_move(state,node.move,id)
            id=1-id
            

        if node.untried_node != []:
            m=random.choice(node.untried_node)
            state=game.make_move(state,m,id)
            id=1-id
            node=node.expand(m,state)

        while not game.is_terminal(state) and game.board_blank(state) != []:
            state=game.make_move(state,move=random.choice(game.board_blank(state)),player_id=id)
            id=1-id

        while node is not None:
            node.backpropagation(game.reward(state))
            node=node.parent
        
        li=[]
        for i in mstnode.childnode:
            li.append(i.value/i.visit)
    return mstnode, mstnode.childnode[np.argmax(li)]
        

In [188]:
game = ConnectFour()
root_state = [[' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' ']]        
mst,best_child=mcts(game=game,begin_state=root_state,iteration=200)
print('mst.value:',mst.value/mst.visit)
for i in mst.childnode:
    print(i.move,':',i.value/i.visit)
print('--------------------------------')
game.show_board(best_child.state)

mst.value: 0.24
1 : -0.3333333333333333
3 : 0.2571428571428571
4 : 0.23529411764705882
6 : -0.3333333333333333
0 : 0.16666666666666666
2 : 0.13043478260869565
5 : 0.45454545454545453
--------------------------------
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', ' ', ' ']


In [3]:
game = ConnectFour()
root_state = [[' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' ']]        
mst,best_child=mcts(game=game,begin_state=root_state,iteration=200)
print('mst.value:',mst.value/mst.visit)
for i in mst.childnode:
    print(i.move,':',i.value/i.visit)
print('--------------------------------')
game.show_board(best_child.state)
while(not game.is_terminal(best_child.state)):
    state=best_child.state
    print('your move:')
    move=int(input())
    new_state=game.make_move(state, move, 1)
    game.show_board(new_state)
    mst,best_child=mcts(game=game,begin_state=new_state,iteration=200)
    print('mst.value:',mst.value/mst.visit)
    for i in mst.childnode:
        print(i.move,':',i.value/i.visit)
    print('--------------------------------')
    game.show_board(best_child.state)

mst.value: 0.12
6 : 0.22727272727272727
5 : -0.047619047619047616
4 : 0.22727272727272727
0 : 0.1794871794871795
3 : -0.14285714285714285
1 : -0.125
2 : 0.09090909090909091
--------------------------------
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', 'X']
your move:


 4


[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', 'O']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', 'X']
mst.value: 0.17
1 : 0.08333333333333333
3 : 0.3448275862068966
5 : 0.1111111111111111
2 : 0.2962962962962963
6 : -0.6666666666666666
0 : -0.058823529411764705
4 : -0.14285714285714285
--------------------------------
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', ' ', 'O']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', 'X']
your move:


 5


[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', ' ', 'O']
[' ', ' ', ' ', ' ', ' ', 'O']
[' ', ' ', ' ', ' ', ' ', 'X']
mst.value: 0.185
1 : 0.25
3 : 0.1724137931034483
5 : 0.23529411764705882
6 : 0.23529411764705882
2 : -0.06666666666666667
4 : 0.05263157894736842
0 : 0.21212121212121213
--------------------------------
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', ' ', 'O']
[' ', ' ', ' ', ' ', ' ', 'O']
[' ', ' ', ' ', ' ', ' ', 'X']
your move:


...

[' ', ' ', ' ', ' ', ' ', 'O']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', 'X', 'O', 'O']
[' ', ' ', ' ', ' ', 'O', 'O']
[' ', ' ', ' ', ' ', ' ', 'X']
mst.value: 0.23
0 : -1.0
6 : -0.3333333333333333
2 : 0.17647058823529413
3 : 0.4639175257731959
4 : -0.3333333333333333
5 : -0.06666666666666667
1 : 0.1875
--------------------------------
[' ', ' ', ' ', ' ', ' ', 'O']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', 'X', 'X']
[' ', ' ', ' ', 'X', 'O', 'O']
[' ', ' ', ' ', ' ', 'O', 'O']
[' ', ' ', ' ', ' ', ' ', 'X']
your move:


 0


[' ', ' ', ' ', ' ', 'O', 'O']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', 'X', 'X']
[' ', ' ', ' ', 'X', 'O', 'O']
[' ', ' ', ' ', ' ', 'O', 'O']
[' ', ' ', ' ', ' ', ' ', 'X']
mst.value: 0.51
4 : 0.375
3 : 0.25
6 : 0.3333333333333333
1 : -0.3333333333333333
5 : 0.7522935779816514
2 : -0.3333333333333333
0 : 0.2222222222222222
--------------------------------
[' ', ' ', ' ', ' ', 'O', 'O']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', 'X', 'X']
[' ', ' ', ' ', 'X', 'O', 'O']
[' ', ' ', ' ', 'X', 'O', 'O']
[' ', ' ', ' ', ' ', ' ', 'X']
your move:


 0


[' ', ' ', ' ', 'O', 'O', 'O']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', 'X', 'X']
[' ', ' ', ' ', 'X', 'O', 'O']
[' ', ' ', ' ', 'X', 'O', 'O']
[' ', ' ', ' ', ' ', ' ', 'X']
mst.value: 0.75
3 : 0.2
6 : 0.1111111111111111
2 : 0.2727272727272727
5 : 1.0
4 : 0.0
1 : 0.0
0 : 0.4444444444444444
--------------------------------
[' ', ' ', ' ', 'O', 'O', 'O']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', 'X', 'X']
[' ', ' ', ' ', 'X', 'O', 'O']
[' ', ' ', 'X', 'X', 'O', 'O']
[' ', ' ', ' ', ' ', ' ', 'X']
